In [1]:
import networkx as nx;
import matplotlib.pyplot as plt
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;

In [2]:
file_summary = open("Results_Summary.csv", "w");

file_summary.write('Instance\t, Budget\t, Rate\t, Nodes\t, Traffick\t, Bottoms\t, Victims\t,');
file_summary.write('Obj_value\t, MaxFlow\t, Int_Traf\t, Int_Bot\t, Int_Vic\t,  Run_Time\n')

num = 10;

m = {};
budget = [0, 4, 8, 16, 24];
rate = [1, 0.75, 0.50];
network = {};


for n in range(num):
    m[n] = {};
    network[n] = 'Network'+str(n+1);
    
    for b in range(len(budget)):
        m[n][b] = {};       
        
        for r in range(len(rate)):
            
            with open(network[n]+'.csv', newline='') as f:
                reader = csv.reader(f);
                row1 = next(reader);
                s = int(row1[0]);
                t = int(row1[1]);
    
                G = nx.DiGraph();
                data = pd.read_csv(network[n]+'.csv',skiprows=1, header=None);
                n_edge = len(data.index+1);
        
                for i in range(n_edge): 
                    G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], trafficker=data.iat[i,5], 
                    bottom=data.iat[i,6], victim=data.iat[i,7]);
                    
                for i,j in G.edges:
                    if i == s:
                        G.edges[i,j]['capacity'] = math.floor(rate[r]*G.edges[i,j]['capacity']);
                    
                A = 0;
                T = 0;
                B = 0;
                V = 0;

                for i,j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        A = A + 1;
                    if G.edges[i,j]['trafficker'] == 1:
                        T = T + 1;
                    elif G.edges[i,j]['bottom'] == 1:
                        B = B + 1;
                    elif G.edges[i,j]['victim'] == 1:
                        V = V + 1;
            
            m[n][b][r] = gp.Model('GenFormulation'+str(n)+str(b)+str(r));
            
            # Variables

            z = m[n][b][r].addVars(G.edges, vtype=GRB.BINARY);

            gamma = m[n][b][r].addVars(G.edges, vtype=GRB.BINARY);

            theta = m[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);

            beta = m[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);

            x = m[n][b][r].addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);

            alpha = m[n][b][r].addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);

            
            # Constraints

            m[n][b][r].addConstr(gp.quicksum(G.edges[i,j]['cost']*gamma[i,j] for i,j in G.edges) <= budget[b]);

            m[n][b][r].addConstr(gp.quicksum(G.edges[i,j]['capacity']*beta[i,j] for i,j in G.edges) <= x[t,s]);

            m[n][b][r].addConstr(alpha[t] - alpha[s] >= 1);

            for i,j in G.edges:
                if G.edges[i,j]['special'] == 1:
                    m[n][b][r].addConstr(x[i,j] - G.edges[i,j]['capacity']*z[i,j] <= 0);
    
                m[n][b][r].addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
    
                m[n][b][r].addConstr(beta[i,j] + gamma[i,j] - theta[i,j] >= 0);
    
                m[n][b][r].addConstr(x[i,j] - G.edges[i,j]['capacity']*(1-gamma[i,j]) <= 0);
    
            m[n][b][r].addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) 
                                  - gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
            
                # Objective
            
            m[n][b][r].setObjective(gp.quicksum(G.edges[i,j]['special']*z[i,j] for i,j in G.edges), GRB.MINIMIZE);
            
            m[n][b][r].update();
            m[n][b][r].setParam("OutputFlag", 0);
            m[n][b][r].optimize();

            file = open('results_'+network[n]+'_b'+str(budget[b])+'_r'+str(rate[r])+'.txt', "w");
            
            file.write('Number of Nodes: %g' % (T+B+V) +'\n');
            file.write('Number of Traffickers: %g' % T +'\n');
            file.write('Number of Bottoms: %g' % B +'\n');
            file.write('Number of victims: %g' % (V) +'\n');
            file.write('Budget: %g' % budget[b] +'\n\n');
            
            if m[n][b][r].status == GRB.status.OPTIMAL:
                obj = m[n][b][r].objVal;
                sol = A - obj;
                file.write('Number of interdicted edges in cal_A: %g' % sol +'\n');
                file.write('Number of uninterdicted edges in cal_A: %g' % m[n][b][r].objVal +'\n');
                file.write('Max-Flow: %g' % x[t, s].x +'\n');
                file.write('Run-time: %f' % m[n][b][r].Runtime + ' sec');
                file.write('\n\n');    
                file.write('Trafficker Capacity: \n');  
                
                for i,j in G.edges:
                    if i==s:
                        file.write(str(i)+', '+str(j)+': '+ str(G.edges[i,j]['capacity']) +'\n');
                
                int_T = 0;
                int_B = 0;
                int_V = 0;
                file.write('Interdiction plan: \n');
                
                for i, j in G.edges: 
                    if gamma[i,j].x > 0.1: 
                        if G.edges[i,j]['trafficker'] == 1:
                            file.write('Trafficker: %s' % j +'\n');
                            int_T = int_T + 1;
                        elif G.edges[i,j]['bottom'] == 1:
                            file.write('Bottom: %s' % j +'\n');
                            int_B = int_B + 1;
                        elif G.edges[i,j]['victim'] == 1:
                            file.write('Victim: %s' % i +'\n');
                            int_V = int_V + 1;
                
                file.write('\n');
                file.write('Victim nodes with flow' +'\n');    
                
                for i, j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        if x[i,j].x > 0.0000001:
                            file.write('victim:'+ str(i)+'\t Flow: %f' % x[i,j].x +'\n');
                
                file_summary.write(network[n]+'_b'+str(budget[b])+'_r'+str(rate[r])+','+str(budget[b])+','
                                   +str(rate[r])+',' +str(T+B+V)+','+str(T)+','+str(B)+','+str(V)+','
                                   +str(m[n][b][r].objVal)+','+str(x[t, s].x)+','+str(int_T)+','
                                   +str(int_B)+','+str(int_V)+','+str(m[n][b][r].Runtime)+'\n');
                                   
            else:
                file.write('No solution'+'\n'+'\n');
                file_summary.write(network[n]+'_b'+str(budget[b])+'_r'+str(rate[r])+','+str(rate[r])+','+str(T+B+V)+','
                                   +str(T)+','+str(B)+','+str(V)+', No Solution\n');
    
            file.write('\n');
            file.close();
            
             
file_summary.close();

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-30
